#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=22809 sha256=3e82cd8a45963de83ce526dc5ad34944a595704043cd18875862c88c968067d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-dn6hso54/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-09-29 19:44:55.501583: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 19:44:57.853353: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 19:44:57.873720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 19:45:05.160563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [5]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [6]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
json_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.h5'
kerasNLP_model = Boston311KerasNLP()
kerasNLP_model.load( json_file, model_file)
kerasNLP_model.predict_date_rang = {'start':'2023-08-27', 'end':tomorrows_datestring}


In [7]:
data = kerasNLP_model.load_data( 'predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:259: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0]


In [8]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [9]:
clean_data.head()

,case_enquiry_id,event,survival_time_hours,type_Abandoned Bicycle,type_Abandoned Building,type_Abandoned Vehicles,type_Abandoned Vehicles - Private Tow,type_Aircraft Noise Disturbance,type_Alert Boston,type_Animal Found,...,ward_number_20,ward_number_21,ward_number_22,ward_number_3,ward_number_4,ward_number_5,ward_number_6,ward_number_7,ward_number_8,ward_number_9
0,NaN,NaN,NaN,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,NaN,NaN,NaN,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,NaN,NaN,NaN,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,NaN,NaN,NaN,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,NaN,NaN,NaN,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
mydata = None
case_data_file = 'case_data.pkl'
mydata = None

X = None

if os.path.exists(case_data_file):
    mydata = pickle.load(open(case_data_file, "rb"))
else:
    mydata = kerasNLP_model.load_data()

    pickle.dump(mydata, open(case_data_file, "wb"))




In [ ]:
mydata['case_enquiry_id']

In [ ]:
mydata = kerasNLP_model.enhance_data(mydata)


In [ ]:
mydata = kerasNLP_model.apply_scenario(mydata)


In [ ]:

mydata = kerasNLP_model.clean_data(mydata)

In [ ]:
print(mydata['case_enquiry_id'])

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [ ]:
#load second file form pickle or from csv
pickle_file2022 = 'dataframe2.pkl'
EXTRA_mydata_FILE_2022 = './cls_and_pooled_embeddings_with_service_id_2022.csv'

X2022 = None

if os.path.exists(pickle_file2022):
    X2022 = pickle.load(open(pickle_file2022, "rb"))
else:
    X2022 = pd.read_csv(EXTRA_mydata_FILE_2022)

    #rename service_request_id to case_enquiry_id
    X2022.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X2022 = X2022[X2022['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X2022['case_enquiry_id'] = X2022['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X2022['cls_embedding'] = X2022['cls_embedding'].apply(literal_eval).apply(np.array)
    X2022['pooled_embedding'] = X2022['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X2022, open(pickle_file2022, "wb"))

In [ ]:
#print information about X2022
print(X.info())
print(X2022.info())

In [ ]:
#concatenate the two dataframes and reindex
df = pd.concat([X, X2022], ignore_index=True)

In [ ]:
df.shape

In [ ]:

# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)

In [ ]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [ ]:
df = df[is_numeric]

In [ ]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [ ]:
df.shape

In [ ]:
mydata.shape

In [ ]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [ ]:
new_mydata.shape

In [ ]:

df, y = kerasNLP_model.split_data(new_mydata)

In [ ]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')

In [ ]:
#list the number of rows in X and y
print(df.shape)
print(y.shape)


In [ ]:
#free all unused dataframes
df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, X2022, new_mydata, is_numeric, mydata]

for data_frame in df_to_delete:
    del data_frame

In [ ]:

#parse CLS embedding column as array
test_acc = kerasNLP_model.predict(df)

In [ ]:
"""
data = kerasNLP_model.load_data( 'predict' )
data = kerasNLP_model.enhance_data( data, 'predict')
clean_data = kerasNLP_model.clean_data_for_prediction( data )

X_predict, y_predict = kerasNLP_model.split_data( clean_data )
y_predict = kerasNLP_model.model.predict(X_predict)
data['survival_prediction'] = y_predict
return data
"""
